# Model Registry Sampler

Explores the MLflow Model Registry API.
Works as both a Databricks and Jupyter notebook.

**Overview**
* Starts clean: deletes all runs and the registered model.
* Trains a model 5 times with different hyperparameters.
* Gets the best model run and registers it as `production`.
* Gets the three next best model runs and registers them as `staging`.
* Loads the production model and runs predictions - using new `models` URI
* Loads the staging model and runs predictions - - using new `models` URI.

**Databricks Issues**
* If we call update_model_version() immediately after create_model_version() without a sleep, the version may not be in `READY` state
  * ERROR: INVALID_STATE: Model version andre_sklearn_registry_test version 1 has invalid status PENDING_REGISTRATION. Expected status is READY.
* This issue do not occur when running open source MLflow with Jupyter.

**Github**
* https://github.com/amesar/mlflow-examples/blob/master/model_registry/Model_Registry_Sampler.html

**Setup**
* Use DBR ML 6.2 which comes with MLflow 1.4.0 installed

Last updated: 2019-12-06

### Setup

In [1]:
import mlflow
import time
print("MLflow Version:",mlflow.version.VERSION)
mlflow.tracking.get_tracking_uri()

MLflow Version: 1.4.0


'file:///Users/ander/git/andre/mlflow-examples/model_registry/mlruns'

In [2]:
import os
is_databricks  = os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None
is_databricks

False

In [3]:
if is_databricks:
    client = mlflow.tracking.MlflowClient()
    dbutils.widgets.text("Nap Time", "2") 
    naptime = int(dbutils.widgets.get("Nap Time"))
    data_path = "/dbfs/tmp/mlflow/wine-quality.csv"
    model_name = "andre_sklearn_registry_sampler"
    experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    print("experiment_name:",experiment_name)
else:
    data_path = "../data/wine-quality-white.csv"
    mlflow.set_tracking_uri("http://localhost:5000")
    client = mlflow.tracking.MlflowClient()
    naptime = 0
    model_name = "sklearn_registry_sampler"
    experiment_name = "sklearn_registry_sampler"
    mlflow.set_experiment(experiment_name)
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
experiment_id, experiment_name, naptime

('2', 'sklearn_registry_sampler', 0)

In [4]:
if is_databricks:
    host_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("browserHostName").get()
    uri = "https://{}/#mlflow/experiments/{}".format(host_name,experiment_id)
    displayHTML("""<b>Experiment URI:</b> <a href="{}">{}</a>""".format(uri,uri))

In [5]:
if is_databricks:
    uri = "https://{}/#mlflow/models/{}".format(host_name,model_name)
    displayHTML("""<b>Registered Model URI:</b> <a href="{}">{}</a>""".format(uri,uri))

In [6]:
runs = client.list_run_infos(experiment_id)
print("#runs:",len(runs))
for info in runs:
    client.delete_run(info.run_id)

#runs: 5


### Define training

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import mlflow
import mlflow.sklearn

In [8]:
data = pd.read_csv(data_path)
train, test = train_test_split(data)
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [9]:
def train(max_depth):
    with mlflow.start_run(run_name="reg_test") as run:
        run_id = run.info.run_uuid
        dt = DecisionTreeRegressor(max_depth=max_depth)
        dt.fit(train_x, train_y)
        predictions = dt.predict(test_x)
        mlflow.log_param("max_depth", max_depth)
        rmse = np.sqrt(mean_squared_error(test_y, predictions))
        mlflow.log_metric("rmse", rmse)
        print(f"{experiment_id} {run_id} {round(rmse,3)}")
        mlflow.sklearn.log_model(dt, "sklearn-model")

### Create runs

In [10]:
max_depths = [1,2,4,5,16]
for x in max_depths:
    train(x)

2 31609993545b4ccfa714674671a450cb 0.814
2 66d0de28bf65485db439d1ff5ab2d161 0.779
2 cb714e90454e4ea689127cde4799bd46 0.753
2 abdadacf27a64e3b82ff9c3c15679518 0.741
2 7f65da46d2b84b40814823a657c1c733 0.808


### Create model versions

In [11]:
runs = client.search_runs(experiment_id,"", order_by=["metrics.rmse asc"])
for run in runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

0.741 {'max_depth': '5'}
0.753 {'max_depth': '4'}
0.779 {'max_depth': '2'}
0.808 {'max_depth': '16'}
0.814 {'max_depth': '1'}


In [12]:
prod_run = runs[:1][0]
round(prod_run.data.metrics['rmse'],3)

0.741

In [13]:
staging_runs = runs[1:4]
none_runs = runs[4:]
len(staging_runs),len(none_runs)

(3, 1)

In [14]:
for run in staging_runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

0.753 {'max_depth': '4'}
0.779 {'max_depth': '2'}
0.808 {'max_depth': '16'}


In [15]:
for run in none_runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

0.814 {'max_depth': '1'}


## Registry

In [16]:
try:
    client.delete_registered_model(model_name)
    print("Deleting model")
except Exception as e:
    print(e)

Deleting model


In [17]:
from mlflow.exceptions import MlflowException, RestException
try:
    registered_model = client.get_registered_model_details(model_name)
    print("Found existing model")
except RestException as e:
    print("Creating new model")
    client.create_registered_model(model_name)
    registered_model = client.get_registered_model_details(model_name)
type(registered_model), registered_model.__dict__

Creating new model


(mlflow.entities.model_registry.registered_model_detailed.RegisteredModelDetailed,
 {'_name': 'sklearn_registry_sampler',
  '_creation_time': 1575667807657,
  '_last_updated_timestamp': 1575667807657,
  '_description': '',
  '_latest_version': []})

### Production model

In [18]:
prod_run.info.artifact_uri

'/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts'

In [19]:
source = f"{prod_run.info.artifact_uri}/sklearn-model"
client.create_model_version(model_name, source, prod_run.info.run_id)
time.sleep(naptime)

In [20]:
versions = client.get_latest_versions(model_name)
len(versions),versions

(0, [])

In [21]:
registered_model = client.get_registered_model_details(model_name)
versions = registered_model.latest_versions
len(versions),versions

(1,
 [<ModelVersionDetailed: creation_timestamp=1575667807691, current_stage='None', description='', last_updated_timestamp=1575667807691, registered_model=<RegisteredModel: name='sklearn_registry_sampler'>, run_id='abdadacf27a64e3b82ff9c3c15679518', source='/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model', status='READY', status_message='', user_id='', version=1>])

In [22]:
versionDetails = client.get_model_version_details(model_name,1)
versionDetails.__dict__

{'_registered_model': <RegisteredModel: name='sklearn_registry_sampler'>,
 '_version': 1,
 '_creation_time': 1575667807691,
 '_last_updated_timestamp': 1575667807691,
 '_description': '',
 '_user_id': '',
 '_current_stage': 'None',
 '_source': '/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model',
 '_run_id': 'abdadacf27a64e3b82ff9c3c15679518',
 '_status': 'READY',
 '_status_message': ''}

NOTE: Without above sleep, we get this error:

RestException: INVALID_STATE: Model version andre_sklearn_registry_test version 1 has invalid status PENDING_REGISTRATION. Expected status is READY.

In [23]:
client.update_model_version(model_name, 1, stage="Production", description="My prod version")

In [24]:
client.get_model_version_stages(model_name,1)

['None', 'Staging', 'Production', 'Archived']

In [25]:
client.get_latest_versions(model_name)

[<ModelVersionDetailed: creation_timestamp=1575667807691, current_stage='Production', description='My prod version', last_updated_timestamp=1575667807768, registered_model=<RegisteredModel: name='sklearn_registry_sampler'>, run_id='abdadacf27a64e3b82ff9c3c15679518', source='/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model', status='READY', status_message='', user_id='', version=1>]

In [26]:
versionDetails = client.get_model_version_details(model_name,1)
versionDetails.__dict__

{'_registered_model': <RegisteredModel: name='sklearn_registry_sampler'>,
 '_version': 1,
 '_creation_time': 1575667807691,
 '_last_updated_timestamp': 1575667807768,
 '_description': 'My prod version',
 '_user_id': '',
 '_current_stage': 'Production',
 '_source': '/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model',
 '_run_id': 'abdadacf27a64e3b82ff9c3c15679518',
 '_status': 'READY',
 '_status_message': ''}

### Staging

In [27]:
def show_versions(versions):
    for v in versions:
        print(f"{v.run_id} {v.version} {v.current_stage} '{v.description}'")

In [28]:
 for j,run in enumerate(staging_runs):
    print(f"==== {j}")
    source = f"{run.info.artifact_uri}/sklearn-model"
    version = client.create_model_version(model_name, source, run.info.run_id)
    #print(version.__dict__)
    time.sleep(naptime)
    versionDetails = client.get_model_version_details(model_name,version.version)
    print(versionDetails.__dict__)
    client.update_model_version(model_name, version.version, stage="Staging", description=f"My staging version {j}")

==== 0
{'_registered_model': <RegisteredModel: name='sklearn_registry_sampler'>, '_version': 2, '_creation_time': 1575667807850, '_last_updated_timestamp': 1575667807850, '_description': '', '_user_id': '', '_current_stage': 'None', '_source': '/usr/local/opt/mlflow/mlruns/2/cb714e90454e4ea689127cde4799bd46/artifacts/sklearn-model', '_run_id': 'cb714e90454e4ea689127cde4799bd46', '_status': 'READY', '_status_message': ''}
==== 1
{'_registered_model': <RegisteredModel: name='sklearn_registry_sampler'>, '_version': 3, '_creation_time': 1575667807888, '_last_updated_timestamp': 1575667807888, '_description': '', '_user_id': '', '_current_stage': 'None', '_source': '/usr/local/opt/mlflow/mlruns/2/66d0de28bf65485db439d1ff5ab2d161/artifacts/sklearn-model', '_run_id': '66d0de28bf65485db439d1ff5ab2d161', '_status': 'READY', '_status_message': ''}
==== 2
{'_registered_model': <RegisteredModel: name='sklearn_registry_sampler'>, '_version': 4, '_creation_time': 1575667807920, '_last_updated_timest

### Manipulate Versions

#### Update Version

In [29]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
cb714e90454e4ea689127cde4799bd46 2 Staging 'My staging version 0'
66d0de28bf65485db439d1ff5ab2d161 3 Staging 'My staging version 1'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


In [30]:
client.update_model_version(model_name, 3, stage='None')

In [31]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
cb714e90454e4ea689127cde4799bd46 2 Staging 'My staging version 0'
66d0de28bf65485db439d1ff5ab2d161 3 None 'My staging version 1'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


#### Delete Version

In [32]:
client.delete_model_version(model_name, 3)

In [33]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
cb714e90454e4ea689127cde4799bd46 2 Staging 'My staging version 0'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


### Execute version methods

In [34]:
versions = client.search_model_versions(f"name='{model_name}'")
show_versions(versions)

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
cb714e90454e4ea689127cde4799bd46 2 Staging 'My staging version 0'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


In [35]:
versions =  client.get_latest_versions(model_name)
show_versions(versions)

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


In [36]:
registered_model = client.get_registered_model_details(model_name)
registered_model.__dict__

{'_name': 'sklearn_registry_sampler',
 '_creation_time': 1575667807657,
 '_last_updated_timestamp': 1575667808000,
 '_description': '',
 '_latest_version': [<ModelVersionDetailed: creation_timestamp=1575667807691, current_stage='Production', description='My prod version', last_updated_timestamp=1575667807768, registered_model=<RegisteredModel: name='sklearn_registry_sampler'>, run_id='abdadacf27a64e3b82ff9c3c15679518', source='/usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model', status='READY', status_message='', user_id='', version=1>,
  <ModelVersionDetailed: creation_timestamp=1575667807920, current_stage='Staging', description='My staging version 2', last_updated_timestamp=1575667807936, registered_model=<RegisteredModel: name='sklearn_registry_sampler'>, run_id='7f65da46d2b84b40814823a657c1c733', source='/usr/local/opt/mlflow/mlruns/2/7f65da46d2b84b40814823a657c1c733/artifacts/sklearn-model', status='READY', status_message='', user_id='', versi

In [37]:
show_versions(registered_model.latest_versions)

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'
7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


In [38]:
versions = client.get_latest_versions(model_name, stages=["Production"])
show_versions(versions)

abdadacf27a64e3b82ff9c3c15679518 1 Production 'My prod version'


In [39]:
versions = client.get_latest_versions(model_name, stages=["Staging"])
show_versions(versions)

7f65da46d2b84b40814823a657c1c733 4 Staging 'My staging version 2'


### Get Model and predict

#### Production model

In [40]:
 data_predict = data.drop(['quality'], axis=1)

In [41]:
model_uri = f"models:/{model_name}/production"
model_uri

'models:/sklearn_registry_sampler/production'

In [42]:
model = mlflow.sklearn.load_model(model_uri)
model

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [43]:
predictions = model.predict(data_predict)
pd.DataFrame(predictions).head(5)

,0
0,5.384615
1,5.384615
2,5.785714
3,5.960674
4,5.960674


#### Staging model

NOTE: Since there are two staging models, apparently the latest one is returned.
This is not documented.

In [44]:
model = mlflow.sklearn.load_model(f"models:/{model_name}/staging")
model

DecisionTreeRegressor(criterion='mse', max_depth=16, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [45]:
predictions = model.predict(data_predict)
pd.DataFrame(predictions).head(5)

,0
0,6.0
1,6.0
2,6.0
3,6.0
4,6.0


### List Registry

In [46]:
def dump(x,indent="  "):
  print("Registered Model")
  for k,v in x.__dict__.items():
    if k == "_latest_version":
      print("  ",k)
      for e in v:
        print("   ModelVersionDetailed:")
        for k2,v2 in e.__dict__.items():
          print("      ",k2,"-",v2)
    else:
      print("  ",k,"-",v)

In [47]:
def list_registry():
  client = mlflow.tracking.MlflowClient()
  lst = client.list_registered_models()
  print("#register_models:",len(lst))
  for e in lst:
    dump(e)
  print("#register_models:",len(lst))

In [48]:
list_registry()

#register_models: 1
Registered Model
   _name - sklearn_registry_sampler
   _creation_time - 1575667807657
   _last_updated_timestamp - 1575667808000
   _description - 
   _latest_version
   ModelVersionDetailed:
       _registered_model - <RegisteredModel: name='sklearn_registry_sampler'>
       _version - 1
       _creation_time - 1575667807691
       _last_updated_timestamp - 1575667807768
       _description - My prod version
       _user_id - 
       _current_stage - Production
       _source - /usr/local/opt/mlflow/mlruns/2/abdadacf27a64e3b82ff9c3c15679518/artifacts/sklearn-model
       _run_id - abdadacf27a64e3b82ff9c3c15679518
       _status - READY
       _status_message - 
   ModelVersionDetailed:
       _registered_model - <RegisteredModel: name='sklearn_registry_sampler'>
       _version - 4
       _creation_time - 1575667807920
       _last_updated_timestamp - 1575667807936
       _description - My staging version 2
       _user_id - 
       _current_stage - Staging
      